# Deep Learning Lab: ECE-00450107
## Meeting 1 - Part 3: The code breaking challenge

Before running the code in this file, make sure that you are **activating the enviourment** in which the following packages are installed.

#### Definitions and Imports:

In [1]:
####################################
## DO NOT EDIT THIS CODE SECTION
from DL_Lab1_functions import *
import albumentations as A # for using augmentations. if not used you can remove this.
warnings.filterwarnings('ignore')
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
matplotlib.use('Agg')

%matplotlib tk
####################################

/opt/anaconda3/envs/ml-env/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


#### Set fixed seeds to enable reproducing the results:

In [2]:
# write the last ID digit of each student in the team
id_digit1 = 319046504
id_digit2 = 206492910
seed = (id_digit1+id_digit2)%10

####################################
## DO NOT EDIT THIS CODE SECTION
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.use_deterministic_algorithms(True)
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.benchmark = False
train_flag = False
####################################

## Code Breaker!

In this task we will build and train from scratch a fully-connected neural classifier on the 'EMNIST Letters' dataset.
<center width="100%"><img src="./assets/emnist.jpeg" width="300px"></center>


Load "EMNIST Letters" dataset and get to know it

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train = torchvision.datasets.EMNIST(root="/usr/share/DL_exp/datasets/emnist-letters", split='letters', train=True, download=True)
test = torchvision.datasets.EMNIST(root="/usr/share/DL_exp/datasets/emnist-letters", split='letters', train=False, download=True)
train_size = len(train)
test_size = len(test)

# Count the number of samples in each category
labels = [label for _, label in train]
label_counts = Counter(labels)
print(f"The train set contains {train_size} images, divided into {len(label_counts.items())} sections:")

labels = [label for _, label in test]
label_counts = Counter(labels)
print(f"The test set contains {test_size} images, divided into {len(label_counts.items())} sections:")
for label, count in label_counts.items():
    print(f"Label {label}: {count} examples")

The train set contains 124800 images, divided into 26 sections:
The test set contains 20800 images, divided into 26 sections:
Label 1: 800 examples
Label 2: 800 examples
Label 3: 800 examples
Label 4: 800 examples
Label 5: 800 examples
Label 6: 800 examples
Label 7: 800 examples
Label 8: 800 examples
Label 9: 800 examples
Label 10: 800 examples
Label 11: 800 examples
Label 12: 800 examples
Label 13: 800 examples
Label 14: 800 examples
Label 15: 800 examples
Label 16: 800 examples
Label 17: 800 examples
Label 18: 800 examples
Label 19: 800 examples
Label 20: 800 examples
Label 21: 800 examples
Label 22: 800 examples
Label 23: 800 examples
Label 24: 800 examples
Label 25: 800 examples
Label 26: 800 examples


Normilize and save data in tensors

In [4]:
# save the data to tensors
# need to remove 1 because the labels are 1-26 and the network expects 0-25
train_data_notNorm = train.data.to(torch.float).to(device)
train_labels = train.targets.to(device)-1 
test_data_notNorm = test.data.to(torch.float).to(device)
test_labels = test.targets.to(device)-1

In [5]:
# get the mean and the std of the train set
train_mean = torch.mean(train_data_notNorm)
train_std = torch.std(train_data_notNorm)
print(f"The mean value is {train_mean}")
print(f"The std value is {train_std}")

# normalize the train and test sets using mean and std
train_data = (train_data_notNorm-train_mean)/train_std
test_data = (test_data_notNorm-train_mean)/train_std

The mean value is 43.917964935302734
The std value is 84.39138793945312


From now on - you are on your own... :-)

In [6]:
# Creating a Fully Connected Neural Network Architeture Class 
class WeWantPrice(nn.Module):
    def __init__(self, input_size: int, hidden_layer_size: int, output_size: int):
        super(WeWantPrice, self).__init__()
        # define the network fully connected layers
        self.fc_layer1 = nn.Linear(input_size, 392)
        self.fc_layer2 = nn.Linear(392, 196)
        self.fc_layer3 = nn.Linear(196, output_size)
        # define a flatten layer
        self.flatten = nn.Flatten()
        # define a sigmoid (activation) layer
        self.activation = nn.Sigmoid()
      
    def forward(self, x):
        # define the input layer, operating on flattaned inputs
        flattened_x = self.flatten(x)
        # define the first layer, using linear operation and then activation
        z1 = self.fc_layer1(flattened_x)
        z2 = self.activation(z1)
        z2 = self.fc_layer2(z2)
        z3 = self.activation(z2)
        # define the output layer
        return self.fc_layer3(z3)

In [7]:
# Defining hyper-parameters
hparams = Hyper_Params()
hparams.train_size = train_size
hparams.lr = 0.05
hparams.batch_size = 30
hparams.epochs = 40

In [8]:
# Set the network model with a hidden layer of size 200 and send to device
model = WeWantPrice(input_size=784, hidden_layer_size=392, output_size=26).to(device)

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=hparams.lr)
# Define the loss criterion
loss_function = nn.CrossEntropyLoss()

In [9]:

if not train_flag:
    print("Training started...")
    train_flag = True
    
    # Start a progress graph and a performance table, for visualization of the trainig process:
    hparams.fig, (hparams.ax1, hparams.ax2) = plt.subplots(2, 1, figsize=(15, 9))
    print_performance_grid(Flag=True)
    # Calculate how many iterations the model trains in each epoch
    iter_num = int(np.ceil(hparams.train_size/hparams.batch_size))
    
    # Set the model to training mode
    model.train()
    start_time = time.time() # time the start of training
    # Training loop:
    for epoch in range(hparams.epochs):
        # for each epoch, do:
        hparams.epoch_accuracy_train = np.zeros(iter_num)
        hparams.epoch_loss_train = np.zeros(iter_num)
        # randomly reshuffle the training and test groups before each new epoch:
        index = torch.randperm(hparams.train_size)
        train_data_perm = train_data[index]
        train_labels_perm = train_labels[index]
        # for each batch, do:
        for i, batch in enumerate(range(0, hparams.train_size, hparams.batch_size)):
            # Get a new batch of images and labels
            data = train_data_perm[batch:batch+hparams.batch_size].to(device) 
            target = train_labels_perm[batch:batch+hparams.batch_size].squeeze().to(device)
            
            # Forward pass
            output = model(data) # Apply the network on the new examples
            loss = loss_function(output, target) # calculate the value of the loss function
            
            # Backward pass - ALWAYS IN THIS ORDER!
            optimizer.zero_grad() # First, delete the gradients from the previous iteration
            loss.backward() # Run backward pass on the loss
            optimizer.step() # Preform an algorithm step (using the optimizer)
    
            # Save the loss and accuracy for the graph visualization
            hparams.epoch_accuracy_train[i] = multi_class_accuracy(output, target.squeeze().to(device))
            hparams.epoch_loss_train[i] = loss.item()
            if(i == 0 and epoch == 0) or ((i+1) == iter_num):
                # Freeze the model in order to evaluate the loss and accuracy on the test set
                model.eval()
                test_out = model(test_data)
                test_loss = loss_function(test_out, test_labels.squeeze()).item()
                test_accuracy = multi_class_accuracy(test_out, test_labels.squeeze())
                print_performance(epoch, i, hparams, test_loss, test_accuracy)
                model.train()
                          
    plt.show()
    print(f"Total training took {time.time() - start_time:.2f} seconds")
    
    print("Training finished.")
else:
    print("Error: Please restart the kernel before running the train again.")

Training started...
| Epoch No. | Iter No. | Train Loss | Train Accuracy | Test Loss | Test Accuracy |
----------------------------------------------------------------------------------
|     0     |    1     |    3.22    |      0.0       |    3.3    |     0.05      |
----------------------------------------------------------------------------------
|     0     |   4160   |    1.89    |      0.45      |   1.15    |     0.66      |
----------------------------------------------------------------------------------
|     1     |   4160   |    0.94    |      0.72      |   0.77    |     0.78      |
----------------------------------------------------------------------------------
|     2     |   4160   |    0.66    |      0.81      |   0.59    |     0.83      |
----------------------------------------------------------------------------------
|     3     |   4160   |    0.51    |      0.85      |   0.48    |     0.86      |
-------------------------------------------------------------------

In [11]:
# Save normalization values for later use
stat_path = "norm_values.txt"
with open(stat_path, "w") as file:
    file.write(f"The mean value is  {train_mean:.2f}\n")
    file.write(f"The std value is {train_std:.2f}\n")

# Save the model for later use
model_path = "letter_classifier.pth"
torch.save(model.state_dict(), model_path)
print(f"Your model achieved {test_accuracy:.2f}% accuracy on the test set.")

print(f"Model saved to {model_path}, normalization values were saved to {stat_path}.")

Your model achieved 0.91% accuracy on the test set.
Model saved to letter_classifier.pth, normalization values were saved to norm_values.txt.
